### Описание концепции

Вы - аналитик данных в крупной компании и получили новый проект от клиента - онлайн-школы программирования X. Этот проект включает в себя работу с данными CRM системы, которая ведет учет статусов лидов и сделок. Ваша задача - очистка и анализ этих данных для повышения эффективности работы школы.f

### Описание данных




#### Contacts
- `Id`: Идентификатор контакта.
- `Contact Owner Name`: Имя лица, ответственного за управление контактом.
- `Created Time`: Дата внесения контакта в базу.
- `Modified Time`: Дата последней модификации контакта.



#### Calls
- `Id`: Уникальный идентификатор для каждого звонка.
- `Call Start Time`: Время начала звонка.
- `Call Owner Name`: Имя лица, ответственного за звонок.
- `CONTACTID`: Уникальный идентификатор контакта.
- `Call Type`: Тип звонка.
- `Call Duration (in seconds)`: Длительность звонка в секундах.
- `Call Status`: Окончательный статус звонка.
- `Dialled Number`: Набранный номер телефона.
- `Outgoing Call Status`: Статус исходящих вызовов.
- `Scheduled in CRM`: Указывает, был ли звонок запланирован через систему CRM.
- `Tag`: Тэг вызова

#### Spend
- `Date`: Дата, указывающая, когда были отслежены показы, клики и расходы на рекламу.
- `Source`: Канал, на котором было показано объявление
- `Campaign`: Кампания, в рамках которой было показано объявление.
- `Impressions`: Количество показов рекламы пользователям.
- `Spend`: Количество денег, потраченных на рекламную кампанию или группу объявлений за указанный период.
- `Clicks`: Количество нажатий пользователей на рекламу.
- `AdGroup`: Подмножество в кампании, содержащее одно или несколько объявлений с одинаковыми целями или настройками.
- `Ad`: Конкретная реклама, показываемая пользователям.

#### Deals
- `Id`: Уникальный идентификатор для каждой сделки.
- `Deal Owner Name`: Имя лица, ответственного за сделку.
- `Created Time`: Метка времени, когда была создана сделка.
- `Course duration`: Длительность курса на который поступает студент
- `Months of study`: Количество месяцев которые отучился студент
- `Closing Date`: Дата закрытия сделки, если применимо.
- `Quality`: Классификация качества сделки, указывающая на ее потенциальный или целевой статус.
- `Stage`: Текущая стадия сделки.
- `Lost Reason`: Причина, по которой сделка была потеряна, если применимо.
- `Page`: Веб-страница или целевая страница, на которой был получен лид.
- `Campaign`: Название или код маркетинговой кампании, связанной со сделкой.
- `SLA`: Время действия соглашения об уровне обслуживания, указывающее на время отклика.
- `Content`: (Ad): Конкретная реклама, показываемая пользователям.
- `Term` (AdGroup): Подмножество в кампании, содержащее одно или несколько объявлений с одинаковыми целями или настройками.
- `Source`: Источник лида.
- `Payment Type`: Тип используемого или ожидаемого способа оплаты.
- `Product`: Конкретный продукт или услуга, связанная со сделкой.
- `Education Type`: Тип образования или обучения.
- `Initial Amount Paid`: Первоначальный платеж клиента.
- `Offer Total Amount`: Общая сумма предложения, представленного клиенту.
- `Contact Name`: Идентификатор контактного лица по сделке.
- `City`: Город, относящийся к клиенту.
- `Level of Deutsch`: Уровень владения немецким языком клиента, если применимо.



Перед началом обработки удалим из файла Deals строку со значением `#REF!`, которое было замечено и является ненужным.

Также удалим пустые строки из файлов Excel, поскольку они могут мешать обработке.

In [1]:
from datetime import time, datetime, timedelta
import re
from collections import defaultdict

import numpy as np
import pandas as pd


In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 250)

In [3]:
path_input = 'E:/_ml/___courses/IT Career Hub/Project/project-20241112'

### Contacts

In [4]:
contacts = pd.read_excel(
    f'{path_input}/Contacts (Done).xlsx',
    parse_dates=['Created Time', 'Modified Time'],
    date_format='%d.%m.%Y %H:%M',
)
print(contacts.shape)
contacts.head()

(18548, 4)


,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,2023-06-27 11:28:00,2023-12-22 13:34:00
1,5805028000000872003,Charlie Davis,2023-07-03 11:31:00,2024-05-21 10:23:00
2,5805028000000889001,Bob Brown,2023-07-02 22:37:00,2023-12-21 13:17:00
3,5805028000000907006,Bob Brown,2023-07-03 05:44:00,2023-12-29 15:20:00
4,5805028000000939010,Nina Scott,2023-07-04 10:11:00,2024-04-16 16:14:00


In [5]:
contacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18548 non-null  int64         
 1   Contact Owner Name  18548 non-null  object        
 2   Created Time        18548 non-null  datetime64[ns]
 3   Modified Time       18548 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 579.8+ KB


### Calls

In [6]:
dtype_types = {
    'CONTACTID': pd.Int64Dtype(),
    'Call Duration (in seconds)': pd.Int64Dtype(),
    'Scheduled in CRM': 'object', # pd.BooleanDtype(),
}
calls = pd.read_excel(
    f'{path_input}/Calls (Done).xlsx',
    dtype=dtype_types,
    parse_dates=['Call Start Time'],
    date_format='%d.%m.%Y %H:%M',
)
print(calls.shape)
calls.head()

(95874, 11)


,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,2023-06-30 08:43:00,John Doe,<NA>,Inbound,171,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,2023-06-30 08:46:00,John Doe,<NA>,Outbound,28,Attended Dialled,NaN,Completed,False,NaN
2,5805028000000764027,2023-06-30 08:59:00,John Doe,<NA>,Outbound,24,Attended Dialled,NaN,Completed,False,NaN
3,5805028000000787003,2023-06-30 09:20:00,John Doe,5805028000000645014,Outbound,6,Attended Dialled,NaN,Completed,False,NaN
4,5805028000000768019,2023-06-30 09:30:00,John Doe,5805028000000645014,Outbound,11,Attended Dialled,NaN,Completed,False,NaN


In [7]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          95874 non-null  int64         
 1   Call Start Time             95874 non-null  datetime64[ns]
 2   Call Owner Name             95874 non-null  object        
 3   CONTACTID                   91941 non-null  Int64         
 4   Call Type                   95874 non-null  object        
 5   Call Duration (in seconds)  95791 non-null  Int64         
 6   Call Status                 95874 non-null  object        
 7   Dialled Number              0 non-null      float64       
 8   Outgoing Call Status        86875 non-null  object        
 9   Scheduled in CRM            86875 non-null  object        
 10  Tag                         0 non-null      float64       
dtypes: Int64(2), datetime64[ns](1), float64(2), int64(1), 

Сразу удалим пустые колонки: `Dialled Number`, `Tag`

In [8]:
calls.drop(columns=['Dialled Number', 'Tag'], inplace=True)

In [9]:
calls['Call Status'].value_counts()

Call Status
Attended Dialled              70703
Unattended Dialled            16030
Missed                         5922
Received                       3077
Overdue                          60
Scheduled Attended Delay         22
Cancelled                        20
Scheduled Unattended Delay       17
Scheduled Attended               14
Scheduled Unattended              6
Scheduled                         3
Name: count, dtype: int64

In [10]:
calls['Outgoing Call Status'].value_counts()

Outgoing Call Status
Completed    86792
Overdue         60
Cancelled       20
Scheduled        3
Name: count, dtype: int64

In [11]:
calls['Call Type'].value_counts()

Call Type
Outbound    86875
Missed       5921
Inbound      3078
Name: count, dtype: int64

In [12]:
calls[(calls['Call Type']=='Missed')&calls['Call Duration (in seconds)']>0]

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM


### Spend

In [13]:
spend = pd.read_excel(f'{path_input}/Spend (Done).xlsx')
print(spend.shape)
spend.head()

(20779, 8)


,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,NaN,NaN
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
3,2023-07-03,Google Ads,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN


In [14]:
spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetime64[ns]
 1   Source       20779 non-null  object        
 2   Campaign     14785 non-null  object        
 3   Impressions  20779 non-null  int64         
 4   Spend        20779 non-null  float64       
 5   Clicks       20779 non-null  int64         
 6   AdGroup      13951 non-null  object        
 7   Ad           13951 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.3+ MB


In [15]:
spend['Source'].value_counts()

Source
Facebook Ads      9732
Tiktok Ads        3066
Youtube Ads       1926
Google Ads        1428
Telegram posts    1003
Bloggers           787
Webinar            766
SMM                614
Organic            518
CRM                355
Test               262
Partnership        234
Offline             61
Radio               27
Name: count, dtype: int64

In [16]:
spend['Campaign'].value_counts()

Campaign
12.07.2023wide_DE              2073
02.07.23wide_DE                1685
04.07.23recentlymoved_DE       1398
youtube_shorts_DE              1223
07.07.23LAL_DE                 1181
03.07.23women                  1171
12.09.23interests_Uxui_DE      1143
15.07.23b_DE                    529
24.09.23retargeting_DE          504
performancemax_eng_DE           355
20.03.2024wide_PL               240
30.11.23wide_DE                 233
05.07.23interests_DE            214
17.03.24wide_AT                 198
07.12.23test_DE                 176
20.03.24interests_WebDev_PL     169
discovery_DE                    166
brand_search_eng_DE             146
08.04.24wide_webinar_DE         133
youtube_shortsin_AT             133
20.05.24interests_DE            130
20.03.24_widde_PL               129
15.03.2024wide_AT               111
15.11.23wide_webinar_DE         107
01.04.23women_PL                103
performancemax_wide_AT           93
05.09.2023wide_DE                88
18.10.23wide_gos_DE

In [17]:
spend['AdGroup'].value_counts()

AdGroup
wide                             5451
recentlymoved                    1442
women                            1274
LAL1                             1220
Com_august                       1073
interest_work_WebDev              733
interest_programming_WebDev       636
b                                 566
retargeting                       504
Com_march                         206
interest_work                     181
Com_july_1                        150
interest_programming              121
promoposts_b                       71
wide_python-developer              56
wide_qa-engineer                   50
berlin_wide                        48
wide_webdesigner                   42
interest_all                       36
interest_python-developer          28
interest_dataanalytics             27
accountant_wide                    21
interest_programming – Copy         8
interest_dataanalytics – Copy       7
Name: count, dtype: int64

### Deals

In [18]:
dtype_types = {
    'Id': pd.Int64Dtype(),
    'Course duration': pd.Int16Dtype(),
    'Months of study': pd.Int16Dtype(),
    'Contact Name': pd.Int64Dtype(),
}

deals = pd.read_excel(
    f'{path_input}/Deals (Done).xlsx',
    dtype=dtype_types,
    parse_dates=['Created Time'],
    date_format='%d.%m.%Y %H:%M',
)
print(deals.shape)
deals.head()

(21593, 23)


,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5805028000056864695,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,women,Facebook Ads,NaN,NaN,NaN,2024-06-21 15:30:00,<NA>,<NA>,NaN,NaN,5805028000056849495,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,NaN,Organic,NaN,Web Developer,Morning,2024-06-21 15:23:00,6,<NA>,0,2000,5805028000056834471,NaN,NaN
2,5805028000056832357,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,21_06_2024,Telegram posts,NaN,NaN,NaN,2024-06-21 14:45:00,<NA>,<NA>,NaN,NaN,5805028000056854421,NaN,NaN
3,5805028000056824246,Eva Kent,21.06.2024,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,01:00:04,bloggersvideo14com,recentlymoved,Facebook Ads,NaN,NaN,NaN,2024-06-21 13:32:00,<NA>,<NA>,NaN,NaN,5805028000056889351,NaN,NaN
4,5805028000056873292,Ben Hall,21.06.2024,D - Non Target,Lost,Non target,/eng,discovery_DE,00:53:12,website,NaN,Google Ads,NaN,NaN,NaN,2024-06-21 13:21:00,<NA>,<NA>,NaN,NaN,5805028000056876176,NaN,NaN


In [19]:
# Преобразование 'Closing Date' в тип datetime
deals['Closing Date'] = pd.to_datetime(deals['Closing Date'], format='%d.%m.%Y', errors='coerce')

In [20]:
deals['Initial Amount Paid'].value_counts()

Initial Amount Paid
1000          2623
0              876
300            188
500             94
350             82
2000            58
11000           38
200             31
11500           25
3500            22
1500            16
€ 3.500,00      16
450             16
5000            14
4000            13
100             12
3000            11
4500            10
400             10
1                3
600              3
1200             2
700              1
9                1
Name: count, dtype: int64

In [21]:
deals['Offer Total Amount'].value_counts()

Offer Total Amount
11000         1860
0              848
11500          394
5000           295
4000           252
3500           133
9000           115
2500            70
2000            63
3000            58
4500            57
€ 2.900,00      20
1200             6
1000             3
1500             3
10000            2
1                2
6500             1
€ 11398,00       1
11111            1
6000             1
Name: count, dtype: int64

Уберем в двух колонках `Initial Amount Paid`, `'Offer Total Amount` - знаки € и преобразуем в целые числа

In [22]:
def clean_euro_and_convert(value):
    if pd.isna(value): return np.nan
    if isinstance(value, str):
        # Удаляем "€", пробелы и заменяем запятую на точку
        cleaned_value = value.replace('€', '').replace(' ', '').replace('.', '').replace(',', '.')
        return pd.to_numeric(cleaned_value, errors='coerce')
    return value

In [23]:
deals['Initial Amount Paid'] = deals['Initial Amount Paid'].apply(clean_euro_and_convert)
deals['Offer Total Amount'] = deals['Offer Total Amount'].apply(clean_euro_and_convert)

deals['Initial Amount Paid'] = deals['Initial Amount Paid'].astype(pd.Int64Dtype())
deals['Offer Total Amount'] = deals['Offer Total Amount'].astype(pd.Int64Dtype())

In [24]:
deals['Offer Total Amount'].value_counts()

Offer Total Amount
11000    1860
0         848
11500     394
5000      295
4000      252
3500      133
9000      115
2500       70
2000       63
3000       58
4500       57
2900       20
1200        6
1000        3
1500        3
10000       2
1           2
6500        1
11398       1
11111       1
6000        1
Name: count, dtype: Int64

In [25]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21593 entries, 0 to 21592
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21593 non-null  Int64         
 1   Deal Owner Name      21564 non-null  object        
 2   Closing Date         14645 non-null  datetime64[ns]
 3   Quality              19340 non-null  object        
 4   Stage                21593 non-null  object        
 5   Lost Reason          16124 non-null  object        
 6   Page                 21593 non-null  object        
 7   Campaign             16067 non-null  object        
 8   SLA                  15533 non-null  object        
 9   Content              14147 non-null  object        
 10  Term                 12454 non-null  object        
 11  Source               21593 non-null  object        
 12  Payment Type         496 non-null    object        
 13  Product              3592 non-n

Было выяснено, что в некоторых строках перепутаны значения `Initial Amount Paid` - `Offer Total Amount`. Проверим это

In [26]:
deals[deals['Initial Amount Paid']>deals['Offer Total Amount']].shape

(58, 23)

Поменяем суммы местами в таких строках, где Initial > Total

In [27]:
mask = deals['Initial Amount Paid'] > deals['Offer Total Amount']
deals.loc[mask, ['Initial Amount Paid', 'Offer Total Amount']] = \
    deals.loc[mask, ['Offer Total Amount', 'Initial Amount Paid']].values


Сейчас у нас `SLA` в виде строк. Преобразуем в целые минуты.

In [28]:
def sla_to_minutes(value):
    if pd.isna(value):
        return np.nan
    
    base_date = datetime(1900, 1, 1)
    if isinstance(value, str):
        try:
            dt = pd.to_datetime(value)
            hours, minutes = dt.hour, dt.minute
        except ValueError:
            print(f"Неверный формат строки: {value}")
            return np.nan
    elif isinstance(value, time):
        hours, minutes = value.hour, value.minute
    elif isinstance(value, datetime):
        
        hours, minutes = value.hour, value.minute
        delta = value - base_date
        hours += (delta.days + 1) * 24
    else:
        print(f"Неподдерживаемый тип: {value}, {type(value)}")
        return np.nan
    
    total_minutes = hours * 60 + minutes
    return total_minutes

In [29]:
deals['SLA Minutes'] = deals['SLA'].apply(sla_to_minutes)
deals['SLA Minutes'] = deals['SLA Minutes'].astype(pd.Int64Dtype())

In [30]:
deals.drop(columns='SLA', inplace=True)

In [31]:
deals['Stage'].value_counts()

Stage
Lost                         15743
Call Delayed                  2248
Registered on Webinar         2072
Payment Done                   858
Waiting For Payment            325
Qualificated                   128
Registered on Offline Day      100
Need to Call - Sales            33
Need To Call                    31
Test Sent                       25
Need a consultation             23
New Lead                         6
Free Education                   1
Name: count, dtype: int64

In [32]:
deals['Lost Reason'].value_counts()

Lost Reason
Doesn't Answer                             4135
Changed Decision                           2146
Duplicate                                  1771
Non target                                 1761
Stopped Answering                          1588
Invalid number                             1481
needs time to think                         655
Expensive                                   626
Conditions are not suitable                 531
Next stream                                 288
Inadequate                                  176
Gutstein refusal                            172
Considering a different direction in IT     148
Not for myself                              145
Does not speak English                      138
Didn't leave an application                 133
Thought for free                            110
Does not know how to use a computer          50
Went to Rivals                               48
The contract did not fit                     21
Refugee                     

In [33]:
deals['Page'].value_counts()

Page
/eng                         5814
eng/digital-marketing        4552
/eng/test                    2996
/workshop                    1162
/webinar                     1130
/                            1082
/direct                      1076
/eng/ux-ui                   1058
/web-developer                658
/email                        462
/pl-eng                       446
/event                        325
/at-eng                       224
/account                      116
eng/web-developer             105
/pl-eng/web-developer          94
/test                          42
/at-end/web-developer          37
/eng/career                    36
/digital-marketing             33
/offer                         27
/specialoffer                  25
/pl-eng/digital-marketing      21
/at-eng/digital-marketing      19
/ppc                           12
/smm                           11
/ux-ui                          9
/at-ru/ux/ui                    7
/course                         7
/page    

In [34]:
deals['Payment Type'].value_counts()

Payment Type
Recurring Payments    350
One Payment           141
Reservation             5
Name: count, dtype: int64

In [35]:
deals['Product'].value_counts()

Product
Digital Marketing      1990
UX/UI Design           1022
Web Developer           575
Find yourself in IT       4
Data Analytics            1
Name: count, dtype: int64

In [36]:
deals['Education Type'].value_counts()

Education Type
Morning    2895
Evening     404
Name: count, dtype: int64

In [37]:
deals['Course duration'].value_counts()

Course duration
11    3012
6      575
Name: count, dtype: Int64

In [38]:
deals['Months of study'].value_counts()

Months of study
6     107
2     104
3      94
4      93
8      83
7      79
1      67
5      64
9      61
11     45
10     42
0       1
Name: count, dtype: Int64

In [39]:
deals[deals['Months of study']>0].shape

(839, 23)

In [40]:
deals['Campaign'].value_counts()

Campaign
performancemax_digitalmarkt_ru_DE    2653
youtube_shorts_DE                    1635
12.07.2023wide_DE                    1575
02.07.23wide_DE                       975
04.07.23recentlymoved_DE              750
03.07.23women                         612
Dis_DE                                581
07.07.23LAL_DE                        542
12.09.23interests_Uxui_DE             531
24.09.23retargeting_DE                479
webinar1604                           343
work_DE                               317
webinar1906                           290
08.04.24wide_webinar_DE               284
blog2_DE                              273
1performancemax_wide_PL               261
BloggerShina_DE                       246
performancemax_eng_DE                 245
brand_search_eng_DE                   168
web2408_DE                            158
bbo_DE                                154
mu_DE                                 151
15.07.23b_DE                          139
performancemax_wide_AT   

In [41]:
deals['Term'].value_counts()

Term
wide                                                                        3675
Com_august                                                                  1529
recentlymoved                                                                757
women                                                                        643
LAL1                                                                         548
retargeting                                                                  479
invitation                                                                   453
interest_work_WebDev                                                         315
interest_programming_WebDev                                                  259
b                                                                            147
it career hub                                                                139
accountant_wide                                                              121
1_day_before           

In [42]:
deals['Source'].value_counts()

Source
Facebook Ads      4850
Google Ads        4226
Organic           2590
Tiktok Ads        2051
SMM               1730
Youtube Ads       1657
CRM               1656
Bloggers          1089
Telegram posts    1001
Webinar            379
Partnership        203
Test               159
Offline              2
Name: count, dtype: int64

### Проверка дубликатов

**contacts**

In [43]:
print('Дубликатов в contacts:', contacts.duplicated().sum())

Дубликатов в contacts: 0


**calls**

In [44]:
cols_calls = list(calls.columns)
cols_calls.remove('Id')

In [45]:
print('Дубликатов в calls:', calls[cols_calls].duplicated().sum())

Дубликатов в calls: 3257


In [46]:
calls.drop_duplicates(subset=cols_calls, inplace=True)

**spend**

In [47]:
spend = spend.sort_values(by=['Date', 'Source'])

In [48]:
print('Дубликатов в spend:', spend.duplicated().sum())

Дубликатов в spend: 917


Мы видим, что есть дубликаты. Но пока непонятно, может ли быть несколько строк в один день. Посмотрим более детально.

In [49]:
spend[(spend.duplicated(keep=False)) & (spend['Clicks'] > 0)].head(6)

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
13828,2024-03-28,SMM,NaN,0,0.0,4,NaN,NaN
13844,2024-03-28,SMM,NaN,0,0.0,4,NaN,NaN
15125,2024-04-11,SMM,NaN,0,0.0,1,NaN,NaN
15129,2024-04-11,SMM,NaN,0,0.0,1,NaN,NaN
16187,2024-04-22,SMM,NaN,0,0.0,1,NaN,NaN
16190,2024-04-22,SMM,NaN,0,0.0,1,NaN,NaN


Проверим, бывают ли в один день одинаковые строки по SMM и Organic, но с разным кол-вом кликов.

In [50]:
# Группируем данные по 'Date' и 'Source' и считаем количество уникальных значений в колонке 'Clicks'
grouped = spend[(spend['Source'].isin(['SMM', 'Organic']))&(spend['Impressions']==0)].groupby(['Date', 'Source']).agg({
    'Clicks': 'nunique',
    'Spend': 'size'  # Подсчет количества строк в каждой группе
}).reset_index()

# Оставляем только те группы, где количество уникальных значений в 'Clicks' больше 1
groups_with_diff_clicks = grouped[grouped['Clicks'] > 1]

groups_with_diff_clicks.head()

,Date,Source,Clicks,Spend
520,2024-03-19,Organic,2,2
521,2024-03-19,SMM,2,2
522,2024-03-20,Organic,2,2
523,2024-03-20,SMM,2,2
524,2024-03-21,Organic,2,2


In [51]:
spend[(spend['Source'].isin(['SMM', 'Organic'])) & (spend['Date'] == '2024-03-19')]

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
13154,2024-03-19,Organic,NaN,0,0.0,201,NaN,NaN
13161,2024-03-19,Organic,NaN,0,0.0,18,NaN,NaN
13155,2024-03-19,SMM,NaN,0,0.0,7,NaN,NaN
13205,2024-03-19,SMM,NaN,0,0.0,72,NaN,NaN


Мы видим, что один день может быть несколько записей по `SMM`, `Organic`, поэтому даже с одинаковым кол-вом кликов в один день, скорее всего, они не являются дубликатами

**deals**

In [52]:
deals = deals.sort_values(by=['Created Time'])

In [53]:
cols_deals = list(deals.columns)
cols_deals.remove('Id')

In [54]:
print('Дубликатов в deals:', deals[cols_deals].duplicated().sum())

Дубликатов в deals: 9


In [55]:
deals[(deals[cols_deals].duplicated(keep=False))].head(6)

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,SLA Minutes
21549,5805028000001355009,Bob Brown,2023-07-08,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-08 11:39:00,<NA>,<NA>,0,0,5805028000001347003,NaN,NaN,<NA>
21548,5805028000001369097,Bob Brown,2023-07-08,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-08 11:39:00,<NA>,<NA>,0,0,5805028000001347003,NaN,NaN,<NA>
21547,5805028000001375082,Kevin Parker,2023-07-08,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-08 13:29:00,<NA>,<NA>,0,0,5805028000001347038,NaN,NaN,<NA>
21546,5805028000001405002,Kevin Parker,2023-07-08,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-08 13:29:00,<NA>,<NA>,0,0,5805028000001347038,NaN,NaN,<NA>
21487,5805028000001575064,Bob Brown,2023-07-12,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-12 19:21:00,<NA>,<NA>,0,0,5805028000001552025,NaN,NaN,<NA>
21486,5805028000001595050,Bob Brown,2023-07-12,E - Non Qualified,Lost,Duplicate,eng/digital-marketing,performancemax_digitalmarkt_ru_DE,_{region_name}_,NaN,Google Ads,NaN,NaN,NaN,2023-07-12 19:21:00,<NA>,<NA>,0,0,5805028000001552025,NaN,NaN,<NA>


Удалим дубликаты

In [56]:
deals.drop_duplicates(cols_deals, inplace=True)

Удалим строки с тестовыми значениями

In [57]:
deals = deals[deals['Page'] != '/test']

### Удаление неактуальных столбцов

Оставим только колонки, которые могут пригодится для дальнейшего анализа и решения задач.

calls

In [58]:
cols_calls_new = ['Call Start Time', 'Call Owner Name', 'CONTACTID', 'Call Duration (in seconds)']
calls_new = calls[cols_calls_new]

Оставим только звонки не меньше 20 секунд

In [59]:
calls_new = calls_new[calls_new['Call Duration (in seconds)']>=20].reset_index(drop=True)

In [60]:
print(calls_new.shape)
calls_new.head()

(37239, 4)


,Call Start Time,Call Owner Name,CONTACTID,Call Duration (in seconds)
0,2023-06-30 08:43:00,John Doe,<NA>,171
1,2023-06-30 08:46:00,John Doe,<NA>,28
2,2023-06-30 08:59:00,John Doe,<NA>,24
3,2023-07-03 13:08:00,Jane Smith,<NA>,40
4,2023-07-04 15:35:00,Jane Smith,5805028000000645014,20


Для spend пока оставим все колонки

deals

In [61]:
cols_deals_new = list(deals.columns)
cols_deals_remove = ['Quality', 'Page', 'Level of Deutsch', 'Term']

cols_deals_new = [col for col in cols_deals_new if col not in cols_deals_remove]

deals_new = deals[cols_deals_new].reset_index(drop=True)


In [62]:
print(deals_new.shape)
deals_new.head()

(21542, 19)


,Id,Deal Owner Name,Closing Date,Stage,Lost Reason,Campaign,Content,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,SLA Minutes
0,5805028000000922001,Bob Brown,2023-07-03,Lost,Refugee,NaN,NaN,Organic,NaN,NaN,NaN,2023-07-03 17:03:00,<NA>,<NA>,0,0,5805028000001009140,NaN,7127
1,5805028000000927004,Bob Brown,2023-07-09,Lost,Does not speak English,03.07.23women,b3,Facebook Ads,NaN,NaN,NaN,2023-07-03 20:17:00,<NA>,<NA>,<NA>,<NA>,5805028000000961001,NaN,<NA>
2,5805028000000945016,Jane Smith,2023-08-29,Lost,Changed Decision,02.07.23wide_DE,b3,Facebook Ads,NaN,NaN,NaN,2023-07-03 20:39:00,<NA>,<NA>,<NA>,<NA>,5805028000000968001,NaN,81781
3,5805028000000948010,Jane Smith,2023-08-29,Lost,needs time to think,03.07.23women,b3,Facebook Ads,NaN,NaN,NaN,2023-07-04 07:10:00,<NA>,<NA>,<NA>,<NA>,5805028000000979006,NaN,<NA>
4,5805028000000970006,Jane Smith,2023-07-04,Lost,Duplicate,03.07.23women,b3,Facebook Ads,NaN,NaN,NaN,2023-07-04 07:10:00,<NA>,<NA>,<NA>,<NA>,5805028000000979006,NaN,<NA>


### Обработка отсутствующих значений

В оставшихся колонках не видно колонок, в которых можно эффективно заполнить пропущенные значение.

Поэтому попытаемся скорректировать и улучшить те колонки, для которых это возможно. 

**Campaign in spend**

Заменим опечатки и копии для сопоставления потом двух таблиц spend + deals

In [63]:
camp_replace_dict = {'20.03.24_widde_PL': '20.03.24wide_PL',
                    '1performancemax_wide_PL':'performancemax_PL',
                    'performancemax_eng_DE':'performancemax_DE',
                    'performancemax_wide_AT':'performancemax_AT',
                    'performancemax_wide_PL':'performancemax_PL',
                    'youtube_shortsin_AT': 'youtube_shorts_AT',
                    'shorts_PL': 'youtube_shorts_PL',
                    '15.04.24LAL_ab__PL':'15.04.24LAL_PL'}
spend['Campaign'] = spend['Campaign'].replace(camp_replace_dict)

adgr_replace_dict = {'interest_programming – Copy': 'interest_programming',
                    'interest_dataanalytics – Copy': 'interest_dataanalytics',
                    'LAL1': 'LAL',}
spend['AdGroup'] = spend['AdGroup'].replace(adgr_replace_dict)


Оставляем только нужные для дальнейшего анализа колонки

In [64]:
cols_spend_new = ['Date', 'Source', 'Campaign', 'Impressions', 'Spend', 'Clicks', 'AdGroup', 'Ad',]

spend_new = spend[cols_spend_new]


In [65]:
spend_new.head()

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
7,2023-07-03,Bloggers,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
5,2023-07-03,Facebook Ads,03.07.23women,187,3.30,6,women,b3
6,2023-07-03,Facebook Ads,03.07.23women,4,0.02,1,women,b1


**Campaign in deals**

Заменим опечатки или копии

In [66]:
camp_replace_dict = {
    '20.03.24_widde_PL': '20.03.24wide_PL',
    'youtube_shortsin_AT': 'youtube_shorts_AT',
    '2005_Lost_DE': 'Lost_DE',
    '15.04.24LAL_ab__PL': '15.04.24LAL_PL',
    'shorts_PL': 'youtube_shorts_PL',
    'performancemax_eng_DE': 'performancemax_DE',
    'performancemax_digitalmarkt_ru_DE': 'performancemax_DE',
    '1performancemax_wide_PL': 'performancemax_PL',
    'performancemax_wide_AT': 'performancemax_AT',    
}
deals_new['Campaign'] = deals_new['Campaign'].replace(camp_replace_dict)


In [67]:
deals_new = deals_new.drop(columns=[
    'Content', # не будет использоваться
])

In [68]:
deals_new.head()

,Id,Deal Owner Name,Closing Date,Stage,Lost Reason,Campaign,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,SLA Minutes
0,5805028000000922001,Bob Brown,2023-07-03,Lost,Refugee,NaN,Organic,NaN,NaN,NaN,2023-07-03 17:03:00,<NA>,<NA>,0,0,5805028000001009140,NaN,7127
1,5805028000000927004,Bob Brown,2023-07-09,Lost,Does not speak English,03.07.23women,Facebook Ads,NaN,NaN,NaN,2023-07-03 20:17:00,<NA>,<NA>,<NA>,<NA>,5805028000000961001,NaN,<NA>
2,5805028000000945016,Jane Smith,2023-08-29,Lost,Changed Decision,02.07.23wide_DE,Facebook Ads,NaN,NaN,NaN,2023-07-03 20:39:00,<NA>,<NA>,<NA>,<NA>,5805028000000968001,NaN,81781
3,5805028000000948010,Jane Smith,2023-08-29,Lost,needs time to think,03.07.23women,Facebook Ads,NaN,NaN,NaN,2023-07-04 07:10:00,<NA>,<NA>,<NA>,<NA>,5805028000000979006,NaN,<NA>
4,5805028000000970006,Jane Smith,2023-07-04,Lost,Duplicate,03.07.23women,Facebook Ads,NaN,NaN,NaN,2023-07-04 07:10:00,<NA>,<NA>,<NA>,<NA>,5805028000000979006,NaN,<NA>


In [69]:
deals_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21542 entries, 0 to 21541
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21542 non-null  Int64         
 1   Deal Owner Name      21513 non-null  object        
 2   Closing Date         14613 non-null  datetime64[ns]
 3   Stage                21542 non-null  object        
 4   Lost Reason          16084 non-null  object        
 5   Campaign             16056 non-null  object        
 6   Source               21542 non-null  object        
 7   Payment Type         495 non-null    object        
 8   Product              3577 non-null   object        
 9   Education Type       3285 non-null   object        
 10  Created Time         21542 non-null  datetime64[ns]
 11  Course duration      3572 non-null   Int16         
 12  Months of study      837 non-null    Int16         
 13  Initial Amount Paid  4145 non-n

В других блокнотах заметили, что по курсу "Web Developer" и "Evening" всего одна запись. Возможно это ошибочная запись, поэтому удалим эту строку

In [70]:
deals_new[(deals_new['Product']=='Web Developer')&(deals_new['Education Type']=='Evening')]

,Id,Deal Owner Name,Closing Date,Stage,Lost Reason,Campaign,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,SLA Minutes
10793,5805028000030164109,Victor Barnes,NaT,Lost,Changed Decision,NaN,Facebook Ads,NaN,Web Developer,Evening,2024-02-05 12:37:00,6,<NA>,2000,2000,5805028000030143280,Nürnberg,407


In [71]:
deals_new = deals_new[~((deals_new['Product'] == 'Web Developer') & (deals_new['Education Type'] == 'Evening'))]

Также удаляем строки с тестовыми значениями

In [72]:
deals_new = deals_new[deals_new['Source'] != 'Test']
spend_new = spend_new[spend_new['Source'] != 'Test']


Отметим сделки которые были оплачены: `Stage`="Payment Done". Также будем фильтровать суммы первого платежа от 10 евро.

In [73]:
deals_new['Initial Amount Paid'] = deals_new['Initial Amount Paid'].fillna(0)

deals_new['Paid'] = deals_new.apply(lambda x: x['Stage']=="Payment Done" and x['Initial Amount Paid']>10, axis=1).astype(int)
deals_new['Paid'] = deals_new['Paid'].fillna(0)

In [74]:
deals_new['Paid'].sum()

833

**Calls**

- удалим строки с пропущенными инентификаторами клиентов, поскольку они не пригодятся для анализа

In [75]:
calls_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37239 entries, 0 to 37238
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Call Start Time             37239 non-null  datetime64[ns]
 1   Call Owner Name             37239 non-null  object        
 2   CONTACTID                   35891 non-null  Int64         
 3   Call Duration (in seconds)  37239 non-null  Int64         
dtypes: Int64(2), datetime64[ns](1), object(1)
memory usage: 1.2+ MB


In [76]:
calls_new = calls_new.dropna()

In [77]:
calls_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35891 entries, 4 to 37238
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Call Start Time             35891 non-null  datetime64[ns]
 1   Call Owner Name             35891 non-null  object        
 2   CONTACTID                   35891 non-null  Int64         
 3   Call Duration (in seconds)  35891 non-null  Int64         
dtypes: Int64(2), datetime64[ns](1), object(1)
memory usage: 1.4+ MB


### Сохранение обработанных данных

Таблицу Contacts не используем для дальнейшего анализа

In [78]:
calls_new.to_pickle('calls_new.pickle')
spend_new.to_pickle('spend_new.pickle')
deals_new.to_pickle('deals_new.pickle')
